In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import ast
import pandas as pd
import os

In [ ]:
path_fi='/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/20k_finnish_predicted.csv'
path_se='/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/20k_swedish_predicted.csv'
path_de='/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/20k_german_predicted.csv'
path_en='/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/20k_english_predicted.csv'
path_da='/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/20k_danish_predicted.csv'

df= pd.read_csv(path_XX, sep='\t', encoding='utf-16') #set to current language

df['predicted_label'] = df['predicted_label'].apply(ast.literal_eval)

In [ ]:
label_names = [
    'Not Categorisable',
    'Conflict & Crisis', #1
    'Migration Flow', #2
    'Host Country Security', #3
    'Host Country Politics', #4
    'Refugee Rights & Advocacy', #5
    'Host Country Resources', #6
    'Host Country Symbolic Discourse' #7
]

filtered_dfs = {}

#Creating separate dataframes for each label
for i, label_name in enumerate(label_names):
    filtered_dfs[label_name] = df[df['predicted_label'].apply(lambda x: x[i] == 1)]

filtered_dfs['No Label'] = df[df['predicted_label'].apply(lambda x: all(i == 0 for i in (x)))]

In [ ]:
concatenated_texts={}

for label in label_names:
    #Excluding all tweets that belong to Not Categorisable or are predicted to no categories from the summarization files
    if label in ['No Label', 'Not Categorisable']:
        continue

    concatenated_texts[label] = ' \n '.join(filtered_dfs[label]['Translation'])


In [ ]:
directory_path = '/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/06_ZIP_V2/XX/' #set to current country

counter_frame = 1

for label, text in concatenated_texts.items():
    print(f"Processing label: {label}")
    chunk_position = 0
    counter_chunk = 1

    label_directory = os.path.join(directory_path, f"L0{counter_frame}")
    os.makedirs(label_directory, exist_ok=True)

    while chunk_position < len(text):

        #Using 19500 characters as an approximation of a suitable chunk size
        end_position = chunk_position + 19500

        # Adjust the chunk size to end on a newline
        if end_position < len(text):
          end_position = text.rfind('\n', chunk_position, end_position) + 1

        if end_position == 0:
                end_position = chunk_position + 19500

        # Extract the chunk and adjust chunk_position
        chunk = text[chunk_position:end_position]
        chunk_position = end_position

        # Define the file path and write the chunk to a file
        filename = f"{counter_chunk}.txt"
        full_path = os.path.join(label_directory, filename)
        with open(full_path, 'w', encoding='utf-8') as file:
            file.write(chunk)

        print(f"Chunk {counter_chunk} written to {full_path}")
        counter_chunk += 1

    counter_frame += 1
